In [1]:
# ============================================================================
# CELL 1: INSTALL DEPENDENCIES (FIXED VERSION)
# ============================================================================
print("🔧 Installing dependencies...")

# Install compatible versions to avoid import errors
!pip install -q langchain==0.1.0
!pip install -q langchain-community
!pip install -q chromadb==0.4.22

# Install sentence-transformers with newer version that's compatible with modern huggingface-hub
!pip install -q sentence-transformers>=2.7.0

!pip install -q pypdf2
!pip install -q pymupdf
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q tqdm

print("✅ All dependencies installed!")

🔧 Installing dependencies...
✅ All dependencies installed!


In [2]:
# ============================================================================
# CELL 2: IMPORTS
# ============================================================================
import os
import sys
import PyPDF2
import fitz  # PyMuPDF for better text extraction
import warnings
from pathlib import Path
from typing import List, Dict, Tuple
from datetime import datetime
from tqdm import tqdm
import json

warnings.filterwarnings('ignore')

# LangChain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

print("✅ All imports successful")
print(f"📅 Processing started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ All imports successful
📅 Processing started: 2025-11-11 21:52:32


In [3]:
# ============================================================================
# CELL 3: CONFIGURATION
# ============================================================================
class Config:
    """Configuration for document processing and embedding creation"""
    
    # Dataset path - UPDATE THIS to match your dataset name
    DATA_PATH = "/kaggle/input/indian-mining-laws-dataset"
    
    # Output paths
    VECTORSTORE_PATH = "/kaggle/working/mining_laws_vectorstore"
    PROCESSED_TEXT_PATH = "/kaggle/working/processed_texts"
    METADATA_PATH = "/kaggle/working/document_metadata.json"
    
    # Embedding configuration
    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
    # Alternative models (uncomment to use):
    # EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"  # Better quality, slower
    # EMBEDDING_MODEL = "BAAI/bge-small-en-v1.5"  # Good balance
    
    # Text chunking parameters
    CHUNK_SIZE = 1000  # Characters per chunk
    CHUNK_OVERLAP = 200  # Overlap between chunks
    
    # Processing options
    USE_PYMUPDF = True  # Better PDF extraction
    SAVE_PROCESSED_TEXT = True  # Save extracted text for inspection
    
    # Device configuration
    DEVICE = 'cuda' if os.system('nvidia-smi') == 0 else 'cpu'

config = Config()
print(f"✅ Configuration loaded")
print(f"📊 Device: {config.DEVICE}")
print(f"📁 Dataset path: {config.DATA_PATH}")

# Create output directories
os.makedirs(config.PROCESSED_TEXT_PATH, exist_ok=True)
os.makedirs(config.VECTORSTORE_PATH, exist_ok=True)

Tue Nov 11 21:52:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             27W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# ============================================================================
# CELL 4: ADVANCED PDF TEXT EXTRACTOR
# ============================================================================
class PDFTextExtractor:
    """Extract text from PDFs using multiple methods"""
    
    def __init__(self, use_pymupdf=True):
        self.use_pymupdf = use_pymupdf
    
    def extract_with_pymupdf(self, pdf_path: str) -> str:
        """Extract text using PyMuPDF (better quality)"""
        try:
            text = ""
            doc = fitz.open(pdf_path)
            
            for page_num in range(len(doc)):
                page = doc[page_num]
                text += page.get_text()
            
            doc.close()
            return text.strip()
        except Exception as e:
            print(f"⚠️  PyMuPDF failed: {e}")
            return ""
    
    def extract_with_pypdf2(self, pdf_path: str) -> str:
        """Extract text using PyPDF2 (fallback)"""
        try:
            text = ""
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                
                for page in pdf_reader.pages:
                    try:
                        text += page.extract_text() + "\n"
                    except:
                        continue
            
            return text.strip()
        except Exception as e:
            print(f"⚠️  PyPDF2 failed: {e}")
            return ""
    
    def extract_text(self, pdf_path: str) -> str:
        """Extract text with fallback methods"""
        # Try PyMuPDF first (better quality)
        if self.use_pymupdf:
            text = self.extract_with_pymupdf(pdf_path)
            if text and len(text) > 100:
                return text
        
        # Fallback to PyPDF2
        text = self.extract_with_pypdf2(pdf_path)
        return text
    
    def clean_text(self, text: str) -> str:
        """Clean extracted text"""
        # Remove excessive whitespace
        text = ' '.join(text.split())
        
        # Remove control characters
        text = ''.join(char for char in text if char.isprintable() or char == '\n')
        
        return text

print("✅ PDFTextExtractor class defined")

✅ PDFTextExtractor class defined


In [5]:
# ============================================================================
# CELL 5: DOCUMENT PROCESSOR WITH EMBEDDINGS
# ============================================================================
class MiningLawsEmbeddingProcessor:
    """Complete document processing and embedding creation pipeline"""
    
    def __init__(self, config):
        self.config = config
        self.pdf_extractor = PDFTextExtractor(use_pymupdf=config.USE_PYMUPDF)
        
        # Initialize embeddings model
        print(f"🔄 Loading embedding model: {config.EMBEDDING_MODEL}")
        self.embeddings = HuggingFaceEmbeddings(
            model_name=config.EMBEDDING_MODEL,
            model_kwargs={'device': config.DEVICE},
            encode_kwargs={'normalize_embeddings': True}
        )
        print("✅ Embedding model loaded")
        
        # Initialize text splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=config.CHUNK_SIZE,
            chunk_overlap=config.CHUNK_OVERLAP,
            separators=["\n\n", "\n", ". ", "! ", "? ", "; ", ": ", " ", ""],
            length_function=len,
            is_separator_regex=False
        )
        
        # Storage for metadata
        self.document_metadata = {}
        self.processing_stats = {
            'total_files': 0,
            'successful': 0,
            'failed': 0,
            'total_chunks': 0,
            'total_chars': 0,
            'total_words': 0
        }
    
    def classify_document(self, filename: str) -> Dict[str, str]:
        """Classify document type and extract metadata from filename"""
        filename_lower = filename.lower()
        
        # Document type classification
        if 'act' in filename_lower and 'report' not in filename_lower:
            doc_type = 'Act'
        elif 'regulation' in filename_lower or 'rule' in filename_lower:
            doc_type = 'Regulation'
        elif 'circular' in filename_lower:
            doc_type = 'Circular'
        elif 'amendment' in filename_lower:
            doc_type = 'Amendment'
        elif 'report' in filename_lower:
            doc_type = 'Report'
        elif 'policy' in filename_lower:
            doc_type = 'Policy'
        elif 'guideline' in filename_lower:
            doc_type = 'Guideline'
        elif 'scheme' in filename_lower:
            doc_type = 'Scheme'
        elif 'development' in filename_lower:
            doc_type = 'Development Plan'
        else:
            doc_type = 'Document'
        
        # Extract year
        import re
        year_match = re.search(r'(19|20)\d{2}', filename)
        year = year_match.group() if year_match else 'Unknown'
        
        # Subject classification
        subject = self._classify_subject(filename_lower)
        
        return {
            'doc_type': doc_type,
            'year': year,
            'subject': subject
        }
    
    def _classify_subject(self, filename: str) -> str:
        """Classify document subject"""
        if 'coal' in filename:
            return 'Coal Mining'
        elif 'atomic' in filename or 'energy' in filename:
            return 'Atomic Minerals'
        elif 'dgms' in filename:
            return 'Safety & Inspection'
        elif 'employment' in filename or 'women' in filename:
            return 'Labour & Employment'
        elif 'pension' in filename or 'provident' in filename:
            return 'Welfare & Benefits'
        elif 'electricity' in filename:
            return 'Electricity & Energy'
        elif 'mineral' in filename and 'development' in filename:
            return 'Mineral Development'
        elif 'vocational' in filename or 'training' in filename:
            return 'Training & Education'
        elif 'court' in filename or 'enquiry' in filename:
            return 'Legal & Judicial'
        else:
            return 'General Mining'
    
    def process_single_document(self, pdf_path: str) -> Tuple[List[Document], Dict]:
        """Process a single PDF document"""
        filename = os.path.basename(pdf_path)
        
        # Extract text
        text = self.pdf_extractor.extract_text(pdf_path)
        
        if not text or len(text) < 100:
            return [], {'error': 'Insufficient text extracted'}
        
        # Clean text
        text = self.pdf_extractor.clean_text(text)
        
        # Get document classification
        classification = self.classify_document(filename)
        
        # Create base metadata
        base_metadata = {
            'source': filename,
            'path': pdf_path,
            'doc_type': classification['doc_type'],
            'year': classification['year'],
            'subject': classification['subject'],
            'char_count': len(text),
            'word_count': len(text.split()),
            'processed_date': datetime.now().isoformat()
        }
        
        # Save processed text if configured
        if self.config.SAVE_PROCESSED_TEXT:
            text_file = os.path.join(
                self.config.PROCESSED_TEXT_PATH,
                filename.replace('.pdf', '.txt')
            )
            with open(text_file, 'w', encoding='utf-8') as f:
                f.write(text)
        
        # Split into chunks
        chunks = self.text_splitter.split_text(text)
        
        # Create Document objects with metadata
        documents = []
        for idx, chunk in enumerate(chunks):
            chunk_metadata = base_metadata.copy()
            chunk_metadata.update({
                'chunk_id': idx,
                'chunk_total': len(chunks),
                'chunk_chars': len(chunk)
            })
            documents.append(Document(page_content=chunk, metadata=chunk_metadata))
        
        # Update stats
        self.processing_stats['total_chunks'] += len(chunks)
        self.processing_stats['total_chars'] += len(text)
        self.processing_stats['total_words'] += len(text.split())
        
        return documents, base_metadata
    
    def process_all_documents(self) -> List[Document]:
        """Process all documents in the dataset"""
        print("\n" + "="*80)
        print("🚀 STARTING DOCUMENT PROCESSING & EMBEDDING CREATION")
        print("="*80)
        
        all_documents = []
        
        # Get all PDF files
        pdf_files = []
        for root, dirs, files in os.walk(self.config.DATA_PATH):
            for file in files:
                if file.endswith('.pdf'):
                    pdf_files.append(os.path.join(root, file))
        
        self.processing_stats['total_files'] = len(pdf_files)
        
        print(f"\n📊 Found {len(pdf_files)} PDF files")
        print(f"📁 Processing from: {self.config.DATA_PATH}")
        print(f"⚙️  Chunk size: {self.config.CHUNK_SIZE}, Overlap: {self.config.CHUNK_OVERLAP}")
        print(f"🔤 Embedding model: {self.config.EMBEDDING_MODEL}\n")
        
        # Process each file with progress bar
        for pdf_path in tqdm(pdf_files, desc="Processing documents"):
            filename = os.path.basename(pdf_path)
            
            try:
                documents, metadata = self.process_single_document(pdf_path)
                
                if documents:
                    all_documents.extend(documents)
                    self.document_metadata[filename] = metadata
                    self.processing_stats['successful'] += 1
                    tqdm.write(f"✅ {filename}: {len(documents)} chunks")
                else:
                    self.processing_stats['failed'] += 1
                    tqdm.write(f"❌ {filename}: Failed to extract text")
                    
            except Exception as e:
                self.processing_stats['failed'] += 1
                tqdm.write(f"❌ {filename}: Error - {str(e)}")
        
        # Print summary
        self._print_processing_summary()
        
        # Save metadata
        self._save_metadata()
        
        return all_documents
    
    def _print_processing_summary(self):
        """Print processing statistics"""
        print("\n" + "="*80)
        print("📊 PROCESSING SUMMARY")
        print("="*80)
        print(f"Total Files Found:      {self.processing_stats['total_files']}")
        print(f"✅ Successfully Processed: {self.processing_stats['successful']}")
        print(f"❌ Failed:                 {self.processing_stats['failed']}")
        print(f"📦 Total Chunks Created:   {self.processing_stats['total_chunks']:,}")
        print(f"📝 Total Characters:       {self.processing_stats['total_chars']:,}")
        print(f"🔤 Total Words:            {self.processing_stats['total_words']:,}")
        print(f"📊 Avg Chunks/Document:    {self.processing_stats['total_chunks'] / max(self.processing_stats['successful'], 1):.1f}")
        print("="*80 + "\n")
    
    def _save_metadata(self):
        """Save document metadata to JSON"""
        metadata_file = self.config.METADATA_PATH
        
        save_data = {
            'processing_stats': self.processing_stats,
            'documents': self.document_metadata,
            'config': {
                'embedding_model': self.config.EMBEDDING_MODEL,
                'chunk_size': self.config.CHUNK_SIZE,
                'chunk_overlap': self.config.CHUNK_OVERLAP
            }
        }
        
        with open(metadata_file, 'w') as f:
            json.dump(save_data, f, indent=2)
        
        print(f"💾 Metadata saved to: {metadata_file}")
    
    def create_embeddings_and_vectorstore(self, documents: List[Document]) -> Chroma:
        """Create embeddings and store in Chroma vector database"""
        print("\n" + "="*80)
        print("🔄 CREATING EMBEDDINGS & VECTOR STORE")
        print("="*80)
        print(f"📦 Total document chunks: {len(documents)}")
        print(f"🔤 Embedding model: {self.config.EMBEDDING_MODEL}")
        print(f"💾 Vector store path: {self.config.VECTORSTORE_PATH}")
        print("\n⏳ This may take several minutes...\n")
        
        # Create vector store with progress tracking
        vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=self.embeddings,
            persist_directory=self.config.VECTORSTORE_PATH,
            collection_name="mining_laws"
        )
        
        print("\n✅ Vector store created successfully!")
        print(f"📊 Total embeddings: {len(documents)}")
        print(f"💾 Saved at: {self.config.VECTORSTORE_PATH}")
        print("="*80 + "\n")
        
        return vectorstore
    
    def load_existing_vectorstore(self) -> Chroma:
        """Load previously created vector store"""
        print("\n📂 Loading existing vector store...")
        
        if not os.path.exists(self.config.VECTORSTORE_PATH):
            raise FileNotFoundError(
                f"Vector store not found at {self.config.VECTORSTORE_PATH}. "
                "Please run document processing first."
            )
        
        vectorstore = Chroma(
            persist_directory=self.config.VECTORSTORE_PATH,
            embedding_function=self.embeddings,
            collection_name="mining_laws"
        )
        
        print("✅ Vector store loaded successfully\n")
        return vectorstore

print("✅ MiningLawsEmbeddingProcessor class defined")

✅ MiningLawsEmbeddingProcessor class defined


In [6]:
# ============================================================================
# CELL 6: PROCESS DOCUMENTS & CREATE EMBEDDINGS (MAIN EXECUTION)
# ============================================================================
print("\n🎯 Initializing processor...")

# Create processor instance
processor = MiningLawsEmbeddingProcessor(config)

# Process all documents
print("\n⚡ Starting document processing pipeline...")
documents = processor.process_all_documents()

# Create embeddings and vector store
vectorstore = processor.create_embeddings_and_vectorstore(documents)

print("\n" + "="*80)
print("🎉 EMBEDDING CREATION COMPLETE!")
print("="*80)
print(f"✅ {len(documents)} document chunks embedded")
print(f"💾 Vector store saved at: {config.VECTORSTORE_PATH}")
print(f"📄 Processed texts saved at: {config.PROCESSED_TEXT_PATH}")
print(f"📊 Metadata saved at: {config.METADATA_PATH}")
print("="*80 + "\n")


🎯 Initializing processor...
🔄 Loading embedding model: sentence-transformers/all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

✅ DGMS_Annual_Report_2014_Eng-05.pdf: 189 chunks
✅ The Bihar Land Reforms Laws(Regulating Mines and Minerals) Validation Act 1969.pdf: 7 chunks


Processing documents: 100%|██████████| 40/40 [00:17<00:00,  2.32it/s]


✅ COAL MINES REGULATION 2017.pdf: 639 chunks

📊 PROCESSING SUMMARY
Total Files Found:      40
✅ Successfully Processed: 40
❌ Failed:                 0
📦 Total Chunks Created:   11,109
📝 Total Characters:       8,134,814
🔤 Total Words:            1,412,306
📊 Avg Chunks/Document:    277.7

💾 Metadata saved to: /kaggle/working/document_metadata.json

🔄 CREATING EMBEDDINGS & VECTOR STORE
📦 Total document chunks: 11109
🔤 Embedding model: sentence-transformers/all-MiniLM-L6-v2
💾 Vector store path: /kaggle/working/mining_laws_vectorstore

⏳ This may take several minutes...



ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



✅ Vector store created successfully!
📊 Total embeddings: 11109
💾 Saved at: /kaggle/working/mining_laws_vectorstore


🎉 EMBEDDING CREATION COMPLETE!
✅ 11109 document chunks embedded
💾 Vector store saved at: /kaggle/working/mining_laws_vectorstore
📄 Processed texts saved at: /kaggle/working/processed_texts
📊 Metadata saved at: /kaggle/working/document_metadata.json



In [7]:
# ============================================================================
# CELL 7: TEST EMBEDDINGS & SIMILARITY SEARCH
# ============================================================================
print("🧪 TESTING EMBEDDINGS WITH SIMILARITY SEARCH")
print("="*80 + "\n")

def test_similarity_search(query: str, k: int = 3):
    """Test similarity search with embeddings"""
    print(f"🔍 Query: '{query}'")
    print(f"📊 Retrieving top {k} similar chunks...\n")
    
    results = vectorstore.similarity_search(query, k=k)
    
    for idx, doc in enumerate(results, 1):
        print(f"[{idx}] Source: {doc.metadata['source']}")
        print(f"    Type: {doc.metadata['doc_type']} | Subject: {doc.metadata['subject']}")
        print(f"    Year: {doc.metadata['year']}")
        print(f"    Chunk: {doc.metadata['chunk_id']}/{doc.metadata['chunk_total']}")
        print(f"    Preview: {doc.page_content[:200]}...")
        print(f"    Relevance: High\n")

# Test queries
test_queries = [
    "What are the safety requirements for coal mines?",
    "minimum age for workers in mines",
    "DGMS annual report findings",
    "employment of women in mining",
]

for query in test_queries:
    test_similarity_search(query, k=3)
    print("-" * 80 + "\n")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


🧪 TESTING EMBEDDINGS WITH SIMILARITY SEARCH

🔍 Query: 'What are the safety requirements for coal mines?'
📊 Retrieving top 3 similar chunks...

[1] Source: DGMS_Annual_Report_2014_Eng-09.pdf
    Type: Report | Subject: Safety & Inspection
    Year: 2014
    Chunk: 415/669
    Preview: . Provisions of the Coal Mines Regulations, 1957, relating to mine working; explosives and shortfiring; haulage; ventilation; precautions against danger from fire, dust, gas and water and of other pro...
    Relevance: High

[2] Source: DGMS_Annual_Report_2014_Eng-09.pdf
    Type: Report | Subject: Safety & Inspection
    Year: 2014
    Chunk: 640/669
    Preview: . Provisions of the Coal Mines Regulations, 1957, relating to mine working; explosives and shortfiring; haulage; ventilation; precautions against danger from fire, dust, gas and water and of other pro...
    Relevance: High

[3] Source: DGMS_Annual_Report_2014_Eng-13.pdf
    Type: Report | Subject: Safety & Inspection
    Year: 2014
    Chunk: 13

In [8]:
# ============================================================================
# CELL 8: EMBEDDING STATISTICS & ANALYSIS
# ============================================================================
def get_embedding_statistics(documents):
    """Get detailed statistics about embeddings"""
    
    stats = {
        'by_doc_type': {},
        'by_subject': {},
        'by_year': {},
        'chunk_size_distribution': {
            'min': float('inf'),
            'max': 0,
            'avg': 0
        }
    }
    
    total_chunks = 0
    
    for doc in documents:
        # By document type
        doc_type = doc.metadata['doc_type']
        stats['by_doc_type'][doc_type] = stats['by_doc_type'].get(doc_type, 0) + 1
        
        # By subject
        subject = doc.metadata['subject']
        stats['by_subject'][subject] = stats['by_subject'].get(subject, 0) + 1
        
        # By year
        year = doc.metadata['year']
        stats['by_year'][year] = stats['by_year'].get(year, 0) + 1
        
        # Chunk size
        chunk_size = doc.metadata['chunk_chars']
        stats['chunk_size_distribution']['min'] = min(stats['chunk_size_distribution']['min'], chunk_size)
        stats['chunk_size_distribution']['max'] = max(stats['chunk_size_distribution']['max'], chunk_size)
        total_chunks += 1
    
    stats['chunk_size_distribution']['avg'] = sum(d.metadata['chunk_chars'] for d in documents) / len(documents)
    
    print("\n" + "="*80)
    print("📊 EMBEDDING STATISTICS")
    print("="*80)
    
    print("\n📑 By Document Type:")
    for dtype, count in sorted(stats['by_doc_type'].items(), key=lambda x: x[1], reverse=True):
        print(f"  {dtype:20s}: {count:4d} chunks")
    
    print("\n🔖 By Subject:")
    for subject, count in sorted(stats['by_subject'].items(), key=lambda x: x[1], reverse=True):
        print(f"  {subject:25s}: {count:4d} chunks")
    
    print("\n📅 By Year:")
    for year, count in sorted(stats['by_year'].items()):
        print(f"  {year}: {count:4d} chunks")
    
    print("\n📏 Chunk Size Distribution:")
    print(f"  Minimum: {stats['chunk_size_distribution']['min']} chars")
    print(f"  Maximum: {stats['chunk_size_distribution']['max']} chars")
    print(f"  Average: {stats['chunk_size_distribution']['avg']:.0f} chars")
    
    print("="*80 + "\n")
    
    return stats

# Get and display statistics
embedding_stats = get_embedding_statistics(documents)


📊 EMBEDDING STATISTICS

📑 By Document Type:
  Report              : 4386 chunks
  Regulation          : 2869 chunks
  Document            : 2528 chunks
  Act                 :  667 chunks
  Scheme              :  495 chunks
  Policy              :  144 chunks
  Development Plan    :   20 chunks

🔖 By Subject:
  Safety & Inspection      : 3950 chunks
  Coal Mining              : 3447 chunks
  General Mining           : 2409 chunks
  Legal & Judicial         :  457 chunks
  Electricity & Energy     :  403 chunks
  Mineral Development      :  270 chunks
  Training & Education     :  122 chunks
  Atomic Minerals          :   46 chunks
  Labour & Employment      :    5 chunks

📅 By Year:
  1948:  718 chunks
  1952:  190 chunks
  1955:  151 chunks
  1957:  250 chunks
  1961:  528 chunks
  1963:   35 chunks
  1966:  144 chunks
  1969:    7 chunks
  1984:   46 chunks
  1985:   62 chunks
  1986:   10 chunks
  1998:  128 chunks
  2008:  726 chunks
  2011:   45 chunks
  2014: 3929 chunks
  2015:

In [9]:
# ============================================================================
# CELL 9: SAVE EMBEDDING INFO FOR LLM INTEGRATION
# ============================================================================
embedding_info = {
    'vectorstore_path': config.VECTORSTORE_PATH,
    'embedding_model': config.EMBEDDING_MODEL,
    'total_documents': len(documents),
    'chunk_size': config.CHUNK_SIZE,
    'chunk_overlap': config.CHUNK_OVERLAP,
    'device': config.DEVICE,
    'created_date': datetime.now().isoformat(),
    'ready_for_llm': True,
    'statistics': {
        'total_chunks': len(documents),
        'total_files_processed': processor.processing_stats['successful'],
        'by_doc_type': embedding_stats['by_doc_type'],
        'by_subject': embedding_stats['by_subject']
    }
}

with open('/kaggle/working/embedding_info.json', 'w') as f:
    json.dump(embedding_info, f, indent=2)

print("✅ Embedding info saved for LLM integration")
print(f"📄 File: /kaggle/working/embedding_info.json")

print("\n" + "="*80)
print("✅ EMBEDDINGS ARE READY TO BE FED TO LLM!")
print("="*80)
print("\n📋 Next Steps:")
print("1. ✅ Document processing complete")
print("2. ✅ Embeddings created and stored")
print("3. ✅ Vector store persisted to disk")
print("4. ⏭️  Ready for LLM integration (RAG pipeline)")
print("\n💡 You can now use these embeddings to:")
print("   - Retrieve relevant context for user queries")
print("   - Feed to LLM for question answering")
print("   - Build a complete RAG system")
print("="*80 + "\n")

✅ Embedding info saved for LLM integration
📄 File: /kaggle/working/embedding_info.json

✅ EMBEDDINGS ARE READY TO BE FED TO LLM!

📋 Next Steps:
1. ✅ Document processing complete
2. ✅ Embeddings created and stored
3. ✅ Vector store persisted to disk
4. ⏭️  Ready for LLM integration (RAG pipeline)

💡 You can now use these embeddings to:
   - Retrieve relevant context for user queries
   - Feed to LLM for question answering
   - Build a complete RAG system

